# Estimating Financial Risk Through Monte Carlo Simulation

## Introduction

Financial risk management is a critical aspect of modern investment strategies. It involves assessing potential risks in financial markets and implementing strategies to mitigate them. One powerful tool used in risk assessment is Monte Carlo simulation.

Monte Carlo simulation is a statistical technique used to model the probability of different outcomes in a process that cannot easily be predicted due to the intervention of random variables. In the context of finance, Monte Carlo simulation can be employed to estimate the potential range of outcomes for various investment portfolios, helping investors and financial analysts make informed decisions.

In this project, we will explore how to implement Monte Carlo simulation using PySpark, a powerful framework for big data processing, to estimate financial risk. By leveraging PySpark's distributed computing capabilities, we can efficiently simulate large portfolios and derive meaningful insights into their risk profiles.


# Importing PySpark Libraries

We import necessary libraries for working with PySpark, including `pyspark`, `os`, and `sys`. Additionally, we import `SparkContext` and `SparkSession` from `pyspark` to initialize and manage the Spark context and session, respectively.


In [ ]:
import pyspark
import os
import sys
from pyspark import SparkContext
from pyspark.sql import SparkSession

# Initializing Spark Session with Custom Configuration

We initialize a Spark session named `chapter_8` using `SparkSession.builder`. We configure the driver memory to use 16 GB with `.config("spark.driver.memory", "16g")`. This ensures that the driver application has sufficient memory for processing tasks.


In [ ]:
spark = SparkSession.builder.config("spark.driver.memory", "16g").appName('chapter_8').getOrCreate()

# Loading Stock Data

We load stock data from multiple CSV files (`ABAX.csv`, `AAME.csv`, `AEPI.csv`) located in the directory "data/stocksA". We use `spark.read.csv()` method, specifying the list of file paths, enabling header and schema inference with `header='true'` and `inferSchema='true'` parameters, respectively. Finally, we display the first two rows of the combined DataFrame using `stocks.show(2)`.


In [ ]:
stocks = spark.read.csv(["data/stocksA/ABAX.csv","data/stocksA/AAME.csv","data/stocksA/AEPI.csv"], header='true', inferSchema='true')

stocks.show(2)

# Extracting Symbol Information

We extract the symbol information from the file paths of the stock data using PySpark's `functions` module. We add a new column "Symbol" to the `stocks` DataFrame, extracting the filename from the file path. Then, we split the filename by "/" to remove the directory path and by "." to extract the symbol. Finally, we display the first two rows of the modified DataFrame.


In [ ]:
from pyspark.sql import functions as fun

stocks = stocks.withColumn("Symbol", fun.input_file_name()).\
  withColumn("Symbol", fun.element_at(fun.split("Symbol", "/"), -1)).\
  withColumn("Symbol", fun.element_at(fun.split("Symbol", "\."), 1))

stocks.show(2)

# Loading and Extracting Symbol Information for Factors Data

We load factor data from multiple CSV files (`ABAX.csv`, `AAME.csv`, `AEPI.csv`) located in the directory "data/stocksA". We use `spark.read.csv()` method, specifying the list of file paths, enabling header and schema inference with `header='true'` and `inferSchema='true'` parameters, respectively. Then, we add a new column "Symbol" to the `factors` DataFrame, extracting the filename from the file path. We split the filename by "/" to remove the directory path and by "." to extract the symbol.


In [ ]:
factors = spark.read.csv(["data/stocksA/ABAX.csv","data/stocksA/AAME.csv","data/stocksA/AEPI.csv"], header='true', inferSchema='true')

factors = factors.withColumn("Symbol", fun.input_file_name()).\
  withColumn("Symbol", fun.element_at(fun.split("Symbol", "/"), -1)).\
  withColumn("Symbol", fun.element_at(fun.split("Symbol", "\."), 1))

# Filtering Stocks Data

We filter the `stocks` DataFrame based on the count of occurrences of each symbol. Using PySpark's `Window` function, we partition the data by the "Symbol" column and calculate the count of occurrences for each symbol. We then filter the DataFrame to include only symbols with counts greater than 260 times 5 plus 10.


In [ ]:
from pyspark.sql import Window

stocks = stocks.withColumn('count', fun.count('Symbol').\
  over(Window.partitionBy('Symbol'))).\
  filter(fun.col('count') > 260*5 + 10)

# Configuring Legacy Time Parser Policy

We configure the Spark SQL session to use the legacy time parser policy by executing the SQL command `spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")`. This ensures compatibility with legacy time parsing behavior in Spark SQL.


In [ ]:
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

# Parsing Date Column

We parse the "Date" column in the `stocks` DataFrame to convert it into a date type. We first convert the column values to timestamps using `to_timestamp()`, specifying the format 'dd-MMM-yy'. Then, we convert the timestamps to dates using `to_date()`. Finally, we display the schema of the modified DataFrame using `stocks.printSchema()`.


In [ ]:
stocks = stocks.withColumn(
    'Date',
    fun.to_date(fun.to_timestamp(fun.col('Date'), 'dd-MMM-yy'))
)

stocks.printSchema()


# Filtering Date Range

We filter the `stocks` DataFrame to include data within the date range from October 23, 2009, to October 23, 2014. We apply two filter conditions using PySpark's `filter()` function: one to include dates greater than or equal to October 23, 2009, and another to include dates less than or equal to October 23, 2014.


In [ ]:
from datetime import datetime

stocks = stocks.filter(fun.col('Date') >= datetime(2009, 10, 23)).\
  filter(fun.col('Date') <= datetime(2014, 10, 23))

# Parsing Date Column for Factors Data and Filtering Date Range

We parse the "Date" column in the `factors` DataFrame to convert it into a date type, similar to the procedure done for the `stocks` DataFrame. Then, we filter the `factors` DataFrame to include data within the date range from October 23, 2009, to October 23, 2014, similar to the procedure applied for the `stocks` DataFrame.


In [ ]:
factors = factors.withColumn(
    'Date',
    fun.to_date(fun.to_timestamp(fun.col('Date'), 'dd-MMM-yy'))
)

factors = factors.filter(fun.col('Date') >= datetime(2009, 10, 23)).\
  filter(fun.col('Date') <= datetime(2014, 10, 23))

# Converting Spark DataFrames to Pandas DataFrames

We convert the `stocks` and `factors` Spark DataFrames to Pandas DataFrames using the `toPandas()` method. This allows for easier manipulation and analysis of the data using Pandas. We display the first 5 rows of the `factors_pd_df` DataFrame to inspect the converted data.


In [ ]:
stocks_pd_df = stocks.toPandas()
factors_pd_df = factors.toPandas()

factors_pd_df.head(5)

# Calculating Rolling Returns

We define a function `my_fun` to calculate the rolling returns for each stock and factor. We group the data by the "Symbol" column and calculate the rolling returns for the "Close" column using a rolling window of `n_steps`. The calculated returns are then stored in `stock_returns` and `factors_returns` DataFrames. Finally, we reset the index and sort the dataframes to ensure consistency in the ordering of rows.


In [ ]:
n_steps = 10

def my_fun(x):
  return ((x.iloc[-1] - x.iloc[0]) / x.iloc[0])

stock_returns = stocks_pd_df.groupby('Symbol').Close.\
  rolling(window=n_steps).apply(my_fun)

factors_returns = factors_pd_df.groupby('Symbol').Close.\
  rolling(window=n_steps).apply(my_fun)

stock_returns = stock_returns.reset_index().\
  sort_values('level_1').\
  reset_index()

factors_returns = factors_returns.reset_index().\
  sort_values('level_1').\
  reset_index()

# Creating DataFrames with Returns

We create new DataFrames `stocks_pd_df_with_returns` and `factors_pd_df_with_returns` by assigning calculated returns to the original DataFrames `stocks_pd_df` and `factors_pd_df`, respectively. For `stocks_pd_df_with_returns`, we assign the calculated stock returns from `stock_returns['Close']` to a new column named "stock_returns". For `factors_pd_df_with_returns`, we assign the calculated factor returns and squared factor returns from `factors_returns['Close']` and `factors_returns['Close']**2` respectively, to new columns named "factors_returns" and "factors_returns_squared".

We then pivot the `factors_pd_df_with_returns` DataFrame to rearrange the data by dates as rows and symbols as columns. The column names are adjusted accordingly. Finally, we reset the index of the DataFrame for consistency and display the first row.


In [ ]:
stocks_pd_df_with_returns = stocks_pd_df.\
  assign(stock_returns=stock_returns['Close'])

factors_pd_df_with_returns = factors_pd_df.\
  assign(factors_returns=factors_returns['Close'],factors_returns_squared=factors_returns['Close']**2)

factors_pd_df_with_returns = factors_pd_df_with_returns.\
  pivot(index='Date', columns='Symbol', values=['factors_returns', 'factors_returns_squared'])

factors_pd_df_with_returns.columns = factors_pd_df_with_returns.\
  columns.to_series().str.join('_').reset_index()[0]

factors_pd_df_with_returns = factors_pd_df_with_returns.reset_index()

print(factors_pd_df_with_returns.head(1))

# Displaying Columns of DataFrame

We print the columns of the `factors_pd_df_with_returns` DataFrame to inspect the column names after the data transformation and manipulation.


In [ ]:
print(factors_pd_df_with_returns.columns)

# Performing Linear Regression Analysis

We merge the `stocks_pd_df_with_returns` and `factors_pd_df_with_returns` DataFrames into `stocks_factors_combined_df` based on the common column "Date". We then select the feature columns from the combined DataFrame.

We drop rows with missing values in both the feature columns and the target column "stock_returns". Then, we define a function `find_ols_coef()` to perform Ordinary Least Squares (OLS) regression for each stock. The function calculates the coefficients for the feature columns.

We apply the `find_ols_coef()` function to each group of stocks and store the coefficients in `coefs_per_stock` DataFrame. Finally, we reset the index and rename the columns for better readability.


In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression

stocks_factors_combined_df = pd.merge(stocks_pd_df_with_returns,
  factors_pd_df_with_returns, how="left", on="Date")

feature_columns = list(stocks_factors_combined_df.columns[-6:])

with pd.option_context('mode.use_inf_as_na', True):
  stocks_factors_combined_df = stocks_factors_combined_df.\
    dropna(subset=feature_columns + ['stock_returns'])

def find_ols_coef(df):
  y = df[['stock_returns']].values
  X = df[feature_columns]
  regr = LinearRegression()
  regr_output = regr.fit(X, y)
  return list(df[['Symbol']].values[0]) + list(regr_output.coef_[0])

coefs_per_stock = stocks_factors_combined_df.groupby('Symbol').apply(find_ols_coef)

coefs_per_stock = pd.DataFrame(coefs_per_stock).reset_index()
coefs_per_stock.columns = ['symbol', 'factor_coef_list']

coefs_per_stock = pd.DataFrame(
  coefs_per_stock.factor_coef_list.tolist(),
  index=coefs_per_stock.index,
  columns = ['Symbol'] + feature_columns
)


# Plotting Kernel Density Estimation (KDE) for Factor Returns

We select samples of factor returns data for a specific symbol (the first unique symbol in this case) from the `factors_returns` DataFrame. We then plot the Kernel Density Estimation (KDE) of the factor returns using `plot.kde()` function from Pandas. This visualization provides insights into the distribution of factor returns for the selected symbol.


In [ ]:
samples = factors_returns.loc[factors_returns.Symbol == factors_returns.Symbol.unique()[0]]['Close']

samples.plot.kde()

# Calculating Correlation Matrix for Factor Returns

We extract samples of factor returns data for the first three unique symbols from the `factors_returns` DataFrame (`f_1`, `f_2`, `f_3`). Then, we print the sizes of the samples to ensure consistency.

We create a DataFrame containing these samples with 1039 data points for each factor, and then calculate the correlation matrix using the `corr()` method. This matrix quantifies the linear relationship between pairs of factor returns, providing insights into their correlation patterns.


In [ ]:
f_1 = factors_returns.loc[factors_returns.Symbol == factors_returns.Symbol.unique()[0]]['Close']
f_2 = factors_returns.loc[factors_returns.Symbol == factors_returns.Symbol.unique()[1]]['Close']
f_3 = factors_returns.loc[factors_returns.Symbol == factors_returns.Symbol.unique()[2]]['Close']

print(f_1.size, len(f_2), f_3.size)

pd.DataFrame({
    'f1': list(f_1)[1:1040],
    'f2': list(f_2)[1:1040],
    'f3': list(f_3)
}).corr()

# Calculating Covariance Matrix and Mean for Factor Returns

We create a DataFrame containing samples of factor returns data for the first three unique symbols from the `factors_returns` DataFrame (`f_1`, `f_2`, `f_3`). Then, we calculate the covariance matrix using the `cov()` method and convert it to a NumPy array.

Additionally, we calculate the mean of factor returns for each factor using the `mean()` method. These calculations provide insights into the variability and central tendency of factor returns.


In [ ]:
factors_returns_cov = pd.DataFrame({
    'f1': list(f_1)[1:1040],
    'f2': list(f_2)[1:1040],
    'f3': list(f_3)
}).cov().to_numpy()

factors_returns_mean = pd.DataFrame({
    'f1': list(f_1)[1:1040],
    'f2': list(f_2)[1:1040],
    'f3': list(f_3)
}).mean()

# Generating Multivariate Normal Distribution

We generate samples from a multivariate normal distribution using the mean vector `factors_returns_mean` and covariance matrix `factors_returns_cov`. This distribution represents the joint distribution of factor returns for the first three factors. The `multivariate_normal()` function from NumPy's random module is used for this purpose.


In [ ]:
from numpy.random import multivariate_normal

multivariate_normal(factors_returns_mean, factors_returns_cov)

# Broadcasting Variables

We broadcast the following variables to all Spark workers for efficient distributed computation:

1. `b_coefs_per_stock`: Broadcasts the coefficients per stock DataFrame (`coefs_per_stock`), containing the coefficients calculated from the linear regression analysis.
2. `b_feature_columns`: Broadcasts the feature columns list (`feature_columns`), containing the names of the features used in the linear regression analysis.
3. `b_factors_returns_mean`: Broadcasts the mean vector of factor returns (`factors_returns_mean`), representing the average returns for each factor.
4. `b_factors_returns_cov`: Broadcasts the covariance matrix of factor returns (`factors_returns_cov`), representing the variability and covariance among factor returns.


In [ ]:
b_coefs_per_stock = spark.sparkContext.broadcast(coefs_per_stock)
b_feature_columns = spark.sparkContext.broadcast(feature_columns)
b_factors_returns_mean = spark.sparkContext.broadcast(factors_returns_mean)
b_factors_returns_cov = spark.sparkContext.broadcast(factors_returns_cov)

# Generating Seeds DataFrame

We generate a DataFrame `seedsDF` containing seeds for random number generation. The number of seeds is determined by the `parallelism` variable, which specifies the level of parallelism. Each seed is generated based on the `base_seed` and incremented by 1 for each parallel execution.

The DataFrame is created with an integer type column using `IntegerType()`. We then repartition the DataFrame into `parallelism` partitions to ensure parallel execution across Spark workers.


In [ ]:
from pyspark.sql.types import IntegerType

parallelism = 1000
num_trials = 1000000
base_seed = 1496

seeds = [b for b in range(base_seed, base_seed + parallelism)]

seedsDF = spark.createDataFrame(seeds, IntegerType())
seedsDF = seedsDF.repartition(parallelism)

# Defining UDF for Calculating Trial Returns

We define a Python function `calculate_trial_return(x)` to calculate trial returns for a given seed `x`. Inside the function, we iterate over a range of `num_trials/parallelism` to perform multiple trials in parallel.

For each trial, we generate a random integer using Python's `random.randint()` function and set the random seed using `seed(x)` to ensure reproducibility. We then generate random factors from a multivariate normal distribution using the mean and covariance matrix of factor returns stored in the broadcast variables `b_factors_returns_mean` and `b_factors_returns_cov`.

Next, we retrieve the coefficients per stock DataFrame from the broadcast variable `b_coefs_per_stock`. We calculate returns per stock by multiplying the coefficients with the random factors and squared random factors. Finally, we sum the returns across all stocks and append the trial return to a list.

The function returns a list containing trial returns for each iteration.

We define a User Defined Function (UDF) `udf_return` using `udf()` function from PySpark's `functions` module to apply the `calculate_trial_return()` function to each row of the DataFrame.


In [ ]:
import random
from numpy.random import seed
from pyspark.sql.types import LongType, ArrayType, DoubleType
from pyspark.sql.functions import udf

def calculate_trial_return(x):
  trial_return_list = []

  for i in range(int(num_trials/parallelism)):
    random_int = random.randint(0, num_trials*num_trials)
    seed(x)
    random_factors = multivariate_normal(b_factors_returns_mean.value, b_factors_returns_cov.value)
    coefs_per_stock_df = b_coefs_per_stock.value
    returns_per_stock = (coefs_per_stock_df[b_feature_columns.value] *
      (list(random_factors) + list(random_factors**2)))
    trial_return_list.append(float(returns_per_stock.sum(axis=1).sum(), b_coefs_per_stock.value.size))

  return trial_return_list

udf_return = udf(calculate_trial_return, ArrayType(DoubleType()))

# Performing Trials for Calculating Returns

We apply the `udf_return` UDF to each row of the `seedsDF` DataFrame to calculate trial returns for each seed. We create a new column "trial_return" containing the trial returns for each seed.

Next, we use the `explode()` function from PySpark's `functions` module to explode the "trial_return" array column into separate rows. This creates a new DataFrame `trials` where each row corresponds to a single trial return.

Finally, we cache the `trials` DataFrame in memory for faster access during subsequent computations.


In [ ]:
from pyspark.sql.functions import col, explode

trials = seedsDF.withColumn("trial_return", udf_return(col("value")))
trials = trials.select('value', explode('trial_return').alias('trial_return'))

trials.cache()

# Analyzing Trial Returns

We first approximate the 5th percentile of trial returns from the `trials` DataFrame using the `approxQuantile()` function. This provides an estimate of the 5th percentile of trial returns with minimal error.

Next, we order the `trials` DataFrame by trial returns in ascending order, select the first 5% of rows representing the lowest returns (limiting to `trials.count()/20` rows), and calculate the average trial return for this subset. This provides insight into the average performance of the worst 5% of trials.


In [ ]:
trials.approxQuantile('trial_return', [0.05], 0.0)

trials.orderBy(col('trial_return').asc()).\
  limit(int(trials.count()/20)).\
  agg(fun.avg(col("trial_return"))).show()

# Visualizing Trial Returns

We convert the `trials` DataFrame to a Pandas DataFrame named `mytrials`. Then, we plot the trial returns using the `plot.line()` function from Pandas. This visualization provides a line plot of trial returns over time or trial index, allowing for analysis of the distribution and behavior of trial returns.


In [ ]:
import pandas

mytrials=trials.toPandas()
mytrials.plot.line()